In [8]:
import pandas as pd
import numpy as np
from functools import partial
from ctcv_helper import print_table
from ctcv_helper import read_dataframe
from ctcv_helper import read_dataframe_ratio

In [9]:
# Read limits from csv files
limits = {}
for model in ['K4', 'K5', 'K6']:
    df = pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_0p5.csv"%model, sep=",", index_col=None)
    df = df.append(pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_1p0.csv"%model, sep=",", index_col=None), ignore_index=True)
    df = df.append(pd.DataFrame.from_csv("Mar17/limits_comb3_%s_cv_1p5.csv"%model, sep=",", index_col=None), ignore_index=True)
    limits[model] = df

In [10]:
# Read the cross sections from csv files
df_xsbr = {}
df_br = {}
df_xsecs = {}
for model in ['K4', 'K5', 'K6']:
    df_xsbr[model]  = pd.DataFrame.from_csv("xsbr_scalings_%s.csv"%model, sep=",", index_col=None)
    df_br[model]    = pd.DataFrame.from_csv("br_scalings_%s.csv"%model,   sep=",", index_col=None)
    df_xsecs[model] = pd.DataFrame.from_csv("xsecs_tH_ttH_%s.csv"%model, sep=",", index_col=None)

In [11]:
# Print pure limits on r
MOD = 'K6'
print " alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5"
print_table([partial(read_dataframe, df=limits[MOD], att='exp')],
             linepat=" %5.4f ")

 alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5
-0.973 -6.000:  0.0132                 
-0.941 -4.000:  0.0356                 
-0.900 -3.000:  0.0708  0.0708         
-0.862 -2.500:  0.1099                 
-0.800 -2.000:  0.1821  0.1821  0.1821 
-0.692 -1.500:  0.3325  0.3325         
-0.640 -1.333:                  0.4185 
-0.610 -1.250:          0.4678         
-0.500 -1.000:  0.6763  0.6763  0.6763 
-0.410 -0.833:                  0.8809 
-0.360 -0.750:          1.0015         
-0.308 -0.667:                  1.1416 
-0.200 -0.500:  1.5073  1.5068  1.5068 
-0.100 -0.333:                  1.9854 
-0.059 -0.250:          2.2520         
-0.027 -0.167:                  2.5498 
 0.000  0.000:  3.3213  3.3213  3.3203 
 0.027  0.167:                  4.3516 
 0.059  0.250:          4.8672         
 0.100  0.333:                  5.1895 
 0.200  0.500:  4.4219  4.4219  4.4219 
 0.308  0.667:                  2.8848 
 0.360  0.750:          2.3262         
 0.410  0.833:                  1.8906 
 

In [12]:
# Print limit * cross section * BR
def xs_limit(ct,cv, df_xsecs, df_limits, limval='exp'):
    # Take the cross section for cv=1.0, ct=ct/cv
    xsec = read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs, att='tot')
    lim  = read_dataframe(ct, cv, df=df_limits, att=limval)
    return float(xsec)*float(lim)

print " alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5"
print_table([partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=limits[MOD], limval='exp')], linepat=" %5.4f ")

 alpha  Ct/CV   CV=0.5  CV=1.0  CV=1.5
-0.973 -6.000:  0.3052                 
-0.941 -4.000:  0.3168                 
-0.900 -3.000:  0.3136  0.3136         
-0.862 -2.500:  0.3141                 
-0.800 -2.000:  0.3104  0.3104  0.3104 
-0.692 -1.500:  0.3068  0.3068         
-0.640 -1.333:                  0.3068 
-0.610 -1.250:          0.3043         
-0.500 -1.000:  0.3002  0.3002  0.3002 
-0.410 -0.833:                  0.2971 
-0.360 -0.750:          0.2929         
-0.308 -0.667:                  0.2884 
-0.200 -0.500:  0.2803  0.2802  0.2802 
-0.100 -0.333:                  0.2686 
-0.059 -0.250:          0.2599         
-0.027 -0.167:                  0.2517 
 0.000  0.000:  0.2467  0.2467  0.2466 
 0.027  0.167:                  0.2665 
 0.059  0.250:          0.2868         
 0.100  0.333:                  0.3081 
 0.200  0.500:  0.3050  0.3050  0.3050 
 0.308  0.667:                  0.2631 
 0.360  0.750:          0.2498         
 0.410  0.833:                  0.2407 
 

#### Now do this again, but store it in a dataframe and write it to a csv file

In [19]:
df_xs_limits = pd.DataFrame()
df_xs_limits['alpha'] = np.sign(limits[MOD].cf*limits[MOD].cv)*limits[MOD].cf**2/(limits[MOD].cf**2+limits[MOD].cv**2)
df_xs_limits['alpha'] = df_xs_limits.alpha.round(4)
df_xs_limits['ratio'] = limits[MOD].cf/limits[MOD].cv
df_xs_limits['ratio'] = df_xs_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup']:
    df_xs_limits[limval] = np.vectorize(partial(xs_limit,
                                               df_xsecs=df_xsecs[MOD],
                                               df_limits=limits[MOD],
                                               limval=limval))(limits[MOD].cf,
                                                                    limits[MOD].cv)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_xs_limits.drop_duplicates(subset='ratio', inplace=True)
df_xs_limits.sort_values(by='ratio', inplace=True)
df_xs_limits.index = range(1,len(df_xs_limits)+1)
df_xs_limits.to_csv("xs_limits_%s.csv"%MOD)
df_xs_limits

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup
1,-0.9730,-6.000,0.161835,0.224257,0.305174,0.430018,0.601100
2,-0.9412,-4.000,0.166428,0.225167,0.316835,0.445883,0.604301
3,-0.9000,-3.000,0.167866,0.223674,0.313587,0.444691,0.609899
4,-0.8621,-2.500,0.165502,0.222384,0.314139,0.445340,0.607698
5,-0.8000,-2.000,0.164333,0.220928,0.310425,0.443562,0.607212
6,-0.6923,-1.500,0.160920,0.217021,0.306801,0.439486,0.605020
7,-0.6400,-1.333,0.159947,0.215877,0.306773,0.439744,0.606508
8,-0.6098,-1.250,0.158383,0.214191,0.304277,0.437358,0.603871
9,-0.5000,-1.000,0.154796,0.210198,0.300226,0.433491,0.601339
10,-0.4098,-0.833,0.152057,0.207139,0.297132,0.431244,0.601178


#### Produce also a csv file with the pure r limits (for cv == 1)

In [17]:
df_r_limits = pd.DataFrame()
df_r_limits['alpha'] = np.sign(limits[MOD].cf*limits[MOD].cv)*limits[MOD].cf**2/(limits[MOD].cf**2+limits[MOD].cv**2)
df_r_limits['alpha'] = df_r_limits.alpha.round(4)
df_r_limits['ratio'] = limits[MOD].cf/limits[MOD].cv
df_r_limits['ratio'] = df_r_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup']:
    df_r_limits[limval] = getattr(limits[MOD], limval)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_r_limits.drop_duplicates(subset='ratio', inplace=True)
df_r_limits.sort_values(by='ratio', inplace=True)
df_r_limits.index = range(1,len(df_xs_limits)+1)
df_r_limits.to_csv("r_limits_%s.csv"%MOD)


In [18]:
df_r_limits

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup
1,-0.9730,-6.000,0.0070,0.0097,0.0132,0.0186,0.0260
2,-0.9412,-4.000,0.0187,0.0253,0.0356,0.0501,0.0679
3,-0.9000,-3.000,0.0379,0.0505,0.0708,0.1004,0.1377
4,-0.8621,-2.500,0.0579,0.0778,0.1099,0.1558,0.2126
5,-0.8000,-2.000,0.0964,0.1296,0.1821,0.2602,0.3562
6,-0.6923,-1.500,0.1744,0.2352,0.3325,0.4763,0.6557
7,-0.6400,-1.333,0.2182,0.2945,0.4185,0.5999,0.8274
8,-0.6098,-1.250,0.2435,0.3293,0.4678,0.6724,0.9284
9,-0.5000,-1.000,0.3487,0.4735,0.6763,0.9765,1.3546
10,-0.4098,-0.833,0.4508,0.6141,0.8809,1.2785,1.7823
